<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2021/blob/master/03_lab/PCA_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy as sp
import numpy as np
import scipy.linalg as sla
import scipy.sparse as sps
import scipy.sparse.linalg as spla

import matplotlib.pyplot as plt
%pylab inline

import pandas as pd

# PCA

We have a object - feature matrix $F$ of size l x n,

For the PCA the main task is to find such weight matrix $W$ such that 

$$
G = FW
$$

and each column-vector of G is a principal component, that maximises the residual variance of the data.

$$
argmax_w \sum_{i=1}^l G^2_{i1} = argmax_w \sum_{i=1}^l (FW)^2_{ij}

$$

where $G$ - matrix of principle components of size l x m, $W$ is transformation matrix of size n x m from old features to new.

Why do we maximis this objective?

Hint:

$$
Var(Z) = E[(Z - E[Z])^2]
$$



Columns of matrix $W$ represent principal axis in the feature space.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
random_state = 0

# Load Digits dataset
X, y = datasets.load_digits(return_X_y=True)

# Split into train/test
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.5, stratify=y,
                     random_state=random_state)


Here we will study the something like mnist dataset - images of numbers from 0 to 9. each of which is size 8x8 pixels

In [ ]:
X.shape, y.shape

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
fig = plt.figure(1,(10,10))
grid = ImageGrid(fig, 111,
                 nrows_ncols=(2,7),
                 axes_pad=0.1,
                 )
for i in range(14):
    image = X[i,:].squeeze().reshape(8,8)
    grid[i].imshow(image,cmap='gray',interpolation='none')

We can combine all the data preporcessing and algorithm we want to fit with sklearn make_pipeline tool. Thats comes very handy once you want to write more maintainable code and will be easy to check for bugs and change.

In [ ]:
dim = len(X[0])
n_classes = len(np.unique(y))

n_neighbors = 3
n_components = 2

# Reduce dimension to 2 with PCA


knn_pca = make_pipeline(StandardScaler(),
                    PCA(n_components=n_components, random_state=1543),
                    KNeighborsClassifier(n_neighbors=n_neighbors))

# Fit the method's model
knn_pca.fit(X_train, y_train)

acc_knn = knn_pca.score(X_test, y_test)

# Embed the data set in 2 dimensions using the fitted model
X_transformed = knn_pca[:-1].transform(X)

# Plot the projected points and show the evaluation score
plt.figure()
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y, s=30, cmap='Set1')
plt.title("{}, KNN (k={})\nTest accuracy = {:.2f}".format("KNN",
                                                          n_neighbors,
                                                          acc_knn))
plt.show()

In [ ]:
def calculate_score(n_neighbors, n_components):
    ### In this function, implement fitting a pipeline
    ### with a given number or neighbors and pca components
    ### on the train data
    ### and evaluating it on the test data.
    
    knn_pca = make_pipeline(StandardScaler(),
                    PCA(n_components=n_components, random_state=1543),
                    KNeighborsClassifier(n_neighbors=n_neighbors))

    ### Return the test score
    # Wait by where is X_train? and X_test? will this function run?
    knn_pca.fit(X_train, y_train)
    return knn_pca.score(X_test, y_test)

plot the dependence of the score on the n_neigbours and n_components

In [ ]:
results = []

neighbors = range(1, 21)
components = range(1, 16)

for n_n in neighbors:
    for n_c in components:
        results.append(calculate_score(n_n, n_c))

x_pos, y_pos = np.meshgrid(components, neighbors)
plt.figure(figsize=(12,6))
plt.contourf(x_pos, y_pos, np.array(results).reshape(x_pos.shape), levels=100);
plt.colorbar()
plt.xlabel("N_components",fontsize=19)
plt.ylabel("N_neigbours",fontsize=19);

### Lets take another dataset of wines and see the effect of the data standatisation

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline
print(__doc__)

# Code source: Tyler Lanigan <tylerlanigan@gmail.com>
#              Sebastian Raschka <mail@sebastianraschka.com>

# License: BSD 3 clause

RANDOM_STATE = 42
FIG_SIZE = (10, 7)

features = pd.DataFrame(load_wine(return_X_y=False)['data'],
                        columns=load_wine(return_X_y=False)['feature_names'])

target = load_wine(return_X_y=False)['target']
features.head(5)

In [ ]:
# Make a train/test split using 30% test size
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size=0.30,
                                                    random_state=RANDOM_STATE)

OK, now when you now how to make pipeline, make pipeline with standard scaler and PCA and just PCA

In [ ]:
# Fit to data and predict using pipelined PCA.
# Fit to data and predict using pipelined PCA.
unscaled_clf = make_pipeline(PCA(n_components=None))
unscaled_clf.fit(X_train, y_train)

# Fit to data and predict using pipelined scaling, PCA.
std_clf = make_pipeline(StandardScaler(), PCA(n_components=None))
std_clf.fit(X_train, y_train)


# Extract PCA from pipeline
pca = unscaled_clf.named_steps['pca']
pca_std = std_clf.named_steps['pca']


In [ ]:
# Show first principal components
print('\nPC 1 without scaling:\n', pca.components_[0])
print('\nPC 1 with scaling:\n', pca_std.components_[0])

# Use PCA without and with scale on X_train data for visualization.
X_train_transformed = pca.transform(X_train)
scaler = std_clf.named_steps['standardscaler']
X_train_std_transformed = pca_std.transform(scaler.transform(X_train))

# visualize standardized vs. untouched dataset with PCA performed
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=FIG_SIZE)


for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):
    ax1.scatter(X_train_transformed[y_train == l, 0],
                X_train_transformed[y_train == l, 1],
                color=c,
                label='class %s' % l,
                alpha=0.5,
                marker=m
                )

for l, c, m in zip(range(0, 3), ('blue', 'red', 'green'), ('^', 's', 'o')):
    ax2.scatter(X_train_std_transformed[y_train == l, 0],
                X_train_std_transformed[y_train == l, 1],
                color=c,
                label='class %s' % l,
                alpha=0.5,
                marker=m
                )

ax1.set_title('Training dataset after PCA')
ax2.set_title('Standardized training dataset after PCA')

for ax in (ax1, ax2):
    ax.set_xlabel('1st principal component')
    ax.set_ylabel('2nd principal component')
    ax.legend(loc='upper right')
    ax.grid()

plt.tight_layout()

plt.show()

In [ ]:
### Plot the variance ratio explained vs number components. Use the availible PCA class methods to do that.

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(range(pca_std.n_components_), pca_std.explained_variance_ratio_)
plt.ylabel("vairance ratio")
plt.xlabel("# components")
plt.grid()

# SVD decomposition

if M is m x n matrix over field K, there is exists factorisation of it:

$$
M = U * S * V^{\dagger}, where
$$
- $U$ - is m x m unitary matrix over K,
- $S$ - is diagonal m x n matrix with non-negative real numbers,
- $V$ - is n x n unitary matrix over K.

The values $s_i$ of matrix S are known as singular values of M.
This decomposition is called Singular Value Decomposition - SVD.

Columns of $U$ anv $V$ are called left and right singular vectors of $M$:
$$
M v = s u, \
M^{\dagger} u = sv
$$

Various application in mathematics and optimisation - pseudo-inverse computation, low rank factorisation, application in solving systems of equations ...

If we define matrix $M$ to be $F$, and 

$$
G = U * S,
$$

we will get full PCA decomposition, where weight matrix $W$ is now $V$.

So, to get first K principal components we will just take first K columns of matrix $S * U$.

#### We can also look at those components in the initial basis of M. To do that we multiply them to the firt K rows of matrix $V^{\dagger}$.


In [ ]:
!wget https://github.com/yandexdataschool/MLatImperial2021/raw/master/03_lab/swisscows_edit.jpg

In [ ]:
from PIL import Image
from matplotlib.pyplot import imread
from skimage import color


img = color.rgb2grey(imread(r'swisscows_edit.jpg'))
img.shape

In [ ]:
imgplot = plt.imshow(img, cmap='Greys_r')

# PCA via SVD for compression

We will use svd from scipy package

In [ ]:
U, s, V_h = sla.svd(img, full_matrices=False)
print(U.shape, s.shape, V_h.shape)

In [ ]:
U, s, V_h = sla.svd(img, full_matrices=False)
pca_1 = (U[:,0]*s[0])[:,np.newaxis].dot(V_h[0,:][np.newaxis,:])
pca_1.shape

In [ ]:
plt.imshow(pca_1, cmap='Greys_r');

#### Now write a function that will return pricipal components from Ith to Jth in intial basis (Hint: look how we have calculated the first component in the initial basis)

In [ ]:
U, s, V_h = sla.svd(img, full_matrices=False)
def PCA(start_component = 0, end_component = 1, U = U, s = s, V_h = V_h):
    num_of_cols = end_component - start_component
    US = (U[:,start_component:end_component]*s[start_component:end_component]).reshape(U.shape[0],num_of_cols)
    return US.dot(V_h[start_component:end_component,:].reshape(num_of_cols,V_h.shape[1]))

In [ ]:
pca_1 = PCA()
pca_1_20 = PCA(end_component=20)
pca_1_50 = PCA(end_component=50)
pca_20_100 = PCA(20, 100)
pca_20_end = PCA(20, 384)
#pca_full = PCA(0, 384)

In [ ]:
plt.figure(figsize=(16, 4))
plt.subplot(1,3,1)
imgplot = plt.imshow(pca_1, cmap='Greys_r')
plt.title("1 PCA")

plt.subplot(1,3,2)
imgplot = plt.imshow(pca_1_20, cmap='Greys_r')
plt.title("1-20 PCA")

plt.subplot(1,3,3)
imgplot = plt.imshow(pca_1_50, cmap='Greys_r')
plt.title("1-50 PCA")

plt.figure(figsize=(16, 4))
plt.subplot(1,2,1)
imgplot = plt.imshow(pca_20_100, cmap='Greys_r')
plt.title("20-100 PCA")

plt.subplot(1,2,2)
imgplot = plt.imshow(pca_20_end, cmap='Greys_r')
plt.title("20-end PCA")

### What do you think 1st PCA component reflects? How do you find, is 1-50 components gives you a good image?

In [ ]:
img.shape

First components reflects the biggest the place where one global objects transfers to another. At this place, the biggest gradient change happen.

Using first components of the image, it is possible to compress it in size ie.

using first K components will give memory gain

$$
\frac{N_{rows} * N_{cols}}{K * (N_{rows} + N_{cols} + 1)}
$$

Interesting and helpful link

[Medium](https://medium.com/@jonathan_hui/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491)